In [42]:
# imports
import pandas as pd
import random

import os

In [43]:
# read in decklist
# WE ARE RUNNING 21 SWAMPS

decklist = pd.read_csv(os.path.join('data', 'gonti,_night_minister.csv'))

sub_ndxs = [0,1,2,3]

decklist.iloc[sub_ndxs,:]

,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana
0,77,"Shizo, Death's Storehouse",NaN,0.0,[],Legendary Land,"{T}: Add {B}.\n{B}, {T}: Target legendary crea...",['B']
1,497,Diabolic Intent,{1}{B},2.0,['B'],Sorcery,"As an additional cost to cast this spell, sacr...",NaN
2,533,Misinformation,{B},1.0,['B'],Instant,Put up to three target cards from an opponent'...,NaN
3,610,Deadly Rollick,{3}{B},4.0,['B'],Instant,"If you control a commander, you may cast this ...",NaN


In [44]:
# reading in text files
# as a heads up you need to delete the last line of this as its 

def read_decklist_txt(path) -> pd.DataFrame:
    with open(path, 'r') as f:
        string = f.read()
        string = string.split('\n')
        # first index is number
        numbers = [i.split(' ')[0] for i in string]

        # read categories

        cats = [i[i.find('['):i.find(']')+1].strip('[]')
                for i in string]
        cats = [i.split(',') for i in cats]

        # even out number of categories
        max_cat_len = max([len(c) for c in cats])
        cats = [i + (['NA'] * (max_cat_len-len(i)))
                for i in cats]

        # read color tags

        tags = [i[i.find('^'):-1].strip('^') for i in string]
        tags = [i.split(',')[0] for i in tags] # drop colors

        # get names
        names = [i[i.find(' ')+1:i.find('[')-1] for i in string]

        columns = sum(
            [['name'], [f'cat_{ndx}' for ndx in range(max_cat_len)],
             ['tags'], ['no']], []
        )

        decklist = pd.DataFrame(
            [sum([[names[ndx]], cats[ndx], [tags[ndx]], [numbers[ndx]]], [])
             for ndx in range(len(string))],
             columns=columns
        )

        return decklist

In [ ]:
def get_decklist_info(card_data:pd.DataFrame, decklist:pd.DataFrame):
    """Map card data to a decklist"""
    return card_data[card_data['name'].isin(decklist['name'])]

decklist = read_decklist_txt(os.path.join('data', 'gonti,_night minister.txt'))
decklist.index.name = 'decklist_ndx'

# read in card_data from json
card_path = os.path.join('data', 'oracle-cards-20250414210533.json')
card_data = pd.read_json(card_path)

# label indices
card_data.index.names = ['scryfall_ndx']

# match global card data to decklist
card_data = get_decklist_info(card_data, decklist).sort_values('name')

# make multidex for merging
card_data.index = pd.MultiIndex.from_tuples(
    [(card_data.index[i], i) 
     for i in range(card_data.shape[0])],
    names=['scryfall_ndx','decklist_ndx']
)

# merge decklist and card_data



# card_data.index = card_data_ndxs




AttributeError: module 'pandas' has no attribute 'join'

In [52]:
decklist

,name_x,cat_0,cat_1,cat_2,tags,no,object,id,oracle_id,multiverse_ids,...,card_faces,produced_mana,tcgplayer_etched_id,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,content_warning,defense
decklist_ndx,,,,,,,,,,,,,,,,,,,,,
0,Abyssal Harvester,Theft,NA,NA,Thematic,1,card,f2e0f538-5825-47e9-883c-3ec6fd5b25ea,68cc5065-0514-4adf-885f-da554e1db00a,[679131],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Access Tunnel,Land,NA,NA,Thematic,1,card,4bef5957-71a4-4fe0-b2ce-dff8e8690bd9,ed9cc560-f30b-4b60-a094-ccf93ed656a7,[696493],...,NaN,[C],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adaptive Omnitool,Draw,DPS Up,NA,Getting,1,card,1603aab8-731c-407a-bb44-625c36ace43e,e91240bc-4a13-4131-9379-5ddac6555f4a,[692177],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ancient Brass Dragon,Finisher,Force Block,NA,Getting,1,card,6430c61b-407d-471d-9012-07f1ddef28aa,7d79fb4e-2ca0-404a-9663-1f5abce6e733,[562994],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arcane Signet,Ramp,NA,NA,Getting,1,card,935644d4-a140-4f24-b8ab-d5e79a682a9b,0bc7f093-bef0-4f1a-852c-4b75ebf54838,[],...,NaN,"[B, G, R, U, W]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Wishclaw Talisman,Political,NA,NA,Getting,1,card,2bcf89c2-32cc-4c26-8fe4-8cf54f5a2675,ca59fbf4-c774-49d3-8630-108c053e01fb,[495897],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,Witch's Cottage,Land,NA,NA,Thematic,1,card,69d0f5bd-ccea-49b2-bd79-ad5e4d850cf5,81c70ae7-3c18-4c9b-8505-e4db9e0e6518,[680756],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,Withering Boon,Protection,NA,NA,Getting,1,card,b87891cd-b457-4dff-8d18-a7eaf6748fc6,6c8f276e-4e7b-4974-ab02-9356cc0ffb2b,[473211],...,NaN,[B],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# add 

decklist['must_keeps'] = False

decklist.loc[decklist['type_line'].str.find('Land') != -1.0, 'must_keeps'] = True

decklist.to_csv(os.path.join('data', 'decklist.csv'), index=False)



KeyError: 'type_line'

In [ ]:
def gen_sampling_ndxs(deck_ndxs:list[int], no_must_keeps:int, no_basics:int) -> list[int]:
    
    return random.sample(deck_ndxs, 99-no_must_keeps-no_basics-1)

def decklist_from_ndxs(decklist:pd.DataFrame, ndxs:list[int]) -> pd.DataFrame:
    return decklist.iloc[ndxs, :]

def add_repeats(repeats:dict, deck_ndxs:list[int]) -> list[int]:
    for key, value in repeats.items():
        deck_ndxs = sum(
            [[key] * value, deck_ndxs], []
        )
    return deck_ndxs


def sample_decklist(decklist, repeats:dict) -> list[int]:

    """Collect a random sampling of your decklist
    decklist: a pandas dataframe representing your decklist
        in a scryfall format
    repeats: a dictionary of specific decklist ndxs 
        of cards you want repeated
        ! in a future update this will be collected 
            from the number attribute in the decklist"""
    

    # drop must keeps from sampling pool
    # and generate list of ndxs

    ndxs = decklist[decklist['must_keeps'] == False].index.tolist()
    
    # randomly sample the deck based on 
    # the number of basic lands and 'must keep' cards

    ndxs = gen_sampling_ndxs(
        deck_ndxs = ndxs, 
        no_must_keeps = decklist[decklist['must_keeps'] == True].shape[0],
        no_basics = 21 # we can solve for this 
    )
    
    # add in 'must keep' cards

    ndxs = sum(
        [decklist[decklist['must_keeps'] == True].index.tolist(), ndxs] , [] 
    )

    # add in cards that have multiples

    ndxs = add_repeats(repeats, ndxs)

    return ndxs




In [ ]:
# sample the following from each decklist

## initial 7 cards
## free mulligan
## mulligan to 6
## mulligan to 5

# for the previous 4 situations draw two cards from the top of the library

# just deck things

In [ ]:
def draw(deck_ndxs:list[int], no_cards:int) -> list[int]:
    return deck_ndxs[0:no_cards+1]

def shuffle(deck_ndxs:list[int]) -> list[int]:
    random.shuffle(deck_ndxs) # does not return original list
    return deck_ndxs

In [ ]:
repeats = {
    # decklist ndx : no_repeats
    36 : 22
}

# save index sets as a parquet from a dataframe

draw(shuffle(sample_decklist(decklist, repeats)), 7)

ValueError: Sample larger than population or is negative

# sampling format

# generating samples

In [ ]:
# constants

no_deck_samples = 1
no_samples_per_deck = 1000 # we will eventually do this to stability
no_mulligans = 3 
no_cards_in_hand = 7
cards_drawn = 2

# get sampling pool

deck = shuffle(sample_decklist(decklist, repeats))

# sample cards by
# shuffling the deck and drawing 9 cards
#   where the first 7 are the initial hand 
#   and the last two are your first two draws

samples = [
    draw(shuffle(deck), no_cards_in_hand+cards_drawn-1) 
    for i in range(no_samples_per_deck*no_mulligans)
]

# generate sampling indices

sampling_multidex = sum(sum([
    [[(m, n, i) for i in range(no_mulligans)]
    for n in range(no_samples_per_deck)]
    for m in range(no_deck_samples)
], []), [])

# generate data column names

column_names = sum(
    [
        [f'card_no_{no}' for no in range(no_cards_in_hand)],
        [f'draw_no_{no}' for no in range(cards_drawn)]
    ], []
)

data = pd.DataFrame(
    samples, 
    index=pd.MultiIndex.from_tuples(
        sampling_multidex, names=['deck_sample', 'deck_sample_no','mulligan_count']
    ),
    columns=column_names
)

# data.to_parquet(os.path.join(
#     'data', 'samples.parquet'
# ))

data

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         36        110   
                           1                      29         37         36   
                           2                     115        119          8   
            1              0                     114         67        115   
                           1                      82        117         37   
...                                              ...        ...        ...   
            998            1                       2         86         71   
                           2                     115         36         36   
            999            0                      88         36          5   
                           1                     103         36         36   
                           2                     104         63         36   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      98         40         10   
                           1                       1         80         77   
                           2                      36        109         88   
            1              0                     107        117         23   
                           1                     107         70        114   
...                                              ...        ...        ...   
            998            1                      44         36         11   
                           2                     114         29         67   
            999            0                      34        104         36   
                           1                      36         40        106   
                           2                      36         36         36   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                      36        105          5  
                           1                      16        105         36  
                           2                      36         36        114  
            1              0                      36         36         86  
                           1                      29         95         55  
...                                              ...        ...        ...  
            998            1                      63         62         30  
                           2                       5         65         36  
            999            0                      21         65         36  
                           1                      86         93         36  
                           2                      23         80         34  

[3000 rows x 9 columns]

In [ ]:
pd.read_parquet(os.path.join('data','samples.parquet'))

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         48         47   
                           1                      36         38         36   
                           2                      15         10         36   
            1              0                       4         35         80   
                           1                      21        105         67   
...                                              ...        ...        ...   
            998            1                      69         35         97   
                           2                      99         35         36   
            999            0                      51        103         97   
                           1                      60         43         13   
                           2                      96         36        106   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         60         83   
                           1                      28         36         68   
                           2                      80         36         73   
            1              0                      33         36        113   
                           1                      36         74         81   
...                                              ...        ...        ...   
            998            1                      36         89         41   
                           2                     102         13         36   
            999            0                      36         36         36   
                           1                     113         47         32   
                           2                      36         29         33   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                     105         36         59  
                           1                      37         59         29  
                           2                      61          7          4  
            1              0                      36         36        106  
                           1                     113         36        114  
...                                              ...        ...        ...  
            998            1                      36         36         80  
                           2                      43         21         36  
            999            0                      36          6         74  
                           1                       9         97         36  
                           2                      81         47         16  

[3000 rows x 9 columns]

In [ ]:
# need to add a way to map card types or whatever to the integers

def get_mappings(decklist:pd.DataFrame, column:str) -> dict:

    return decklist.loc[:,column].to_dict()


for col in data.columns:
    data[col] = data[col].map(get_mappings(decklist, 'type_line'))

data

card_no_0  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                                         Enchantment   
                           1                         Creature — Phyrexian Horror   
                           2                                   Creature — Spirit   
            1              0                                             Instant   
                           1                                             Sorcery   
...                                                                          ...   
            998            1                                             Instant   
                           2                                   Creature — Spirit   
            999            0                                                Land   
                           1                        Artifact — Equipment // Land   
                           2               Legendary Creature — Aetherborn Rogue   

                                                         card_no_1  \
deck_sample deck_sample_no mulligan_count                            
0           0              0                           Enchantment   
                           1                    Basic Land — Swamp   
                           2               Creature — Dauthi Rogue   
            1              0               Creature — Human Wizard   
                           1                  Artifact — Equipment   
...                                                            ...   
            998            1                                  Land   
                           2                           Enchantment   
            999            0                           Enchantment   
                           1                           Enchantment   
                           2                              Artifact   

                                                      card_no_2  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                        Enchantment   
                           2                            Instant   
            1              0                  Creature — Spirit   
                           1                 Basic Land — Swamp   
...                                                         ...   
            998            1                        Land — Gate   
                           2                        Enchantment   
            999            0                               Land   
                           1                        Enchantment   
                           2                        Enchantment   

                                                      card_no_3  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                            Sorcery   
                           2                        Enchantment   
            1              0                 Enchantment — Aura   
                           1                 Enchantment — Aura   
...                                                         ...   
            998            1               Artifact — Equipment   
                           2                            Instant   
            999            0                            Instant   
                           1                        Enchantment   
                           2                        Enchantment   

                                                                       card_no_4  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                            Creature — Dauthi Minion   
                           1                                   Creature

In [ ]:
# label card types

# lands


for col in data.columns:
    data.loc[data[col].str.find('Land') != -1, col] = 'land'
    data.loc[data[col].str.find('Creature') != -1, col] = 'creature'
    data.loc[data[col].str.find('Enchantment') != -1, col] = 'enchantment'
    data.loc[data[col].str.find('Equipment') != -1, col] = 'equipment' 
    data.loc[data[col].str.find('Sorcery') != -1, col] = 'sorcery'
    data.loc[data[col].str.find('Instant') != -1, col] = 'instant'
    data.loc[data[col].str.find('Artifact') != -1, col] = 'artifact'

# 

In [ ]:
# sum number of lands

categories = []

for col in data.columns:
    categories = sum(
        [[i for i in pd.unique(data[col]).tolist() 
         if i not in categories],categories],[]
    )

categories

['enchantment',
 'creature',
 'instant',
 'sorcery',
 'land',
 'artifact',
 'equipment']